In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
from torch_geometric.data import DataLoader, Dataset, Data
import lightning.pytorch as pl
import seaborn as sns
import pandas as pd
import os
from tqdm import tqdm
import torch
import itertools
import yaml
from pytorch_lightning.loggers import WandbLogger

import matplotlib.pyplot as plt

from epic_clustering.utils import plot_clusters, get_cluster_pos
from epic_clustering.models import MemberClassification

## 1. Load Model and Test Dataloading

In [ ]:
input_dir = "/global/cfs/cdirs/m3443/data/PowerWeek/train/train"
csv_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.csv')]

In [ ]:
events = pd.read_csv(csv_files[0])
event = events[events["event"] == events["event"].unique()[0]]

In [ ]:
event

In [ ]:
# Sorted event.E - get first 40
high_energy_hits = event.sort_values(by="E", ascending=False).iloc[:40]

In [ ]:
# use torch meshgrid to get all pairs between high_energy_hits.hit_number and event.hit_number
pairs = torch.meshgrid(torch.from_numpy(high_energy_hits.hit_number.values), torch.from_numpy(event.hit_number.values))
# convert into a 2 x N array
pairs = torch.stack(pairs).reshape(2, -1).T

In [ ]:
class EventDataset(Dataset):
    """
    The custom default dataset to load CSV events off the disk
    """

    def __init__(self, input_dir, num_events = None, hparams=None, transform=None, pre_transform=None, pre_filter=None, **kwargs):
        super().__init__(input_dir, transform, pre_transform, pre_filter)
        
        self.input_dir = input_dir
        self.hparams = hparams
        self.num_events = num_events
        self.scales = {
                    "E": 30.,
                    "T": 100.,
                    "posx": 200.,
                    "posy": 200.,
                    "posz": 500.,
                }
        
        self.csv_events = self.load_datafiles_in_dir(self.input_dir, self.num_events)

        print("Converting to PyG data objects")
        self.pyg_events = [self.convert_to_pyg(event[1]) for event in tqdm(self.csv_events)]
        
    def load_datafiles_in_dir(self, input_dir, num_events):

        # Each file is 1000 events, so need to load num_events//1000 + 1 files
        csv_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.csv')][:num_events//1000 + 1]
        events = pd.concat([pd.read_csv(f) for f in csv_files])
        if num_events is not None:
            events = events[events.entry < num_events]

        self.scale_features(events)

        return list(events.groupby('entry'))

    def convert_to_pyg(self, event):

        # Convert to PyG data object
        event = event.reset_index(drop=True)
        event = event.drop(columns=['entry'])

        data.edge_index = self.create_training_pairs(event)
        y = event.clusterID[data.edge_index[0]] == event.clusterID[data.edge_index[1]]
        node_features = torch.from_numpy(event[['posx', 'posy', 'posz', 'E']].to_numpy())
        edge_features = torch.cat([node_features[data.edge_index[0]], node_features[data.edge_index[1]]], dim=1)

        data = Data(
                        x = edge_features,
                        y = y
                    )

        data.num_nodes = data.x.shape[0]

        return data
        
    def len(self):
        return len(self.pyg_events)

    def get(self, idx):

        return self.pyg_events[idx]

    def scale_features(self, event):
        """
        Handle feature scaling for the event
        """

        for feature in self.scales.keys():
            event[feature] = event[feature]/self.scales[feature]

        return event

    def create_training_pairs(self, event):
        """
        Create the true edge list for the event. This is 
        """

        # Sorted event.E - get first 40
        high_energy_hits = event.sort_values(by="E", ascending=False).iloc[:40]

        # use torch meshgrid to get all pairs between high_energy_hits.hit_number and event.hit_number
        pairs = torch.meshgrid(torch.from_numpy(high_energy_hits.hit_number.values), torch.from_numpy(event.hit_number.values))
        # convert into a 2 x N array
        pairs = torch.stack(pairs).reshape(2, -1)

        return pairs

In [ ]:
dataset = EventDataset(input_dir, num_events=100)

In [ ]:
# Split dataset into train val and test
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

## 3. Training Loop Test

In [10]:
with open("member_classification.yaml") as f:
    member_classification_config = yaml.safe_load(f)
model = MemberClassification(member_classification_config)
model.setup(stage="fit")

Converting to PyG data objects


100%|██████████| 2110/2110 [00:14<00:00, 142.09it/s]

Loaded 2000 training events, 100 validation events and 10 testing events


In [7]:
event = model.trainset[1]

In [ ]:
logger = WandbLogger(project=member_classification_config["project"])
trainer = pl.Trainer(devices=1, accelerator="gpu", max_epochs=100, logger=logger)
trainer.fit(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: murnanedaniel. Use `wandb login --relogin` to force relogin


/global/homes/d/danieltm/.conda/envs/powerweek/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /global/homes/d/danieltm/.conda/envs/powerweek/lib/p ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/global/homes/d/danieltm/.conda/envs/powerweek/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /global/homes/d/danieltm/.conda/envs/powerweek/lib/p ...
  rank_zero_warn(
You are using a CUDA device ('NVIDIA A100-

Sanity Checking: 0it [00:00, ?it/s]

/global/homes/d/danieltm/.conda/envs/powerweek/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/global/homes/d/danieltm/.conda/envs/powerweek/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  2.23it/s]

/global/homes/d/danieltm/.conda/envs/powerweek/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('purity', ...)` in your `validation_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


/global/homes/d/danieltm/.conda/envs/powerweek/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/global/homes/d/danieltm/.conda/envs/powerweek/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|██████████| 2000/2000 [04:24<00:00,  7.56it/s, v_num=pcsz]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 2000/2000 [04:27<00:00,  7.47it/s, v_num=pcsz]  
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 2000/2000 [04:28<00:00,  7.45it/s, v_num=pcsz]  
Validation: 0it [00:00, ?it/s]
Epoch 3: 100%|██████████| 2000/2000 [04:28<00:00,  7.45it/s, v_num=pcsz]  
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████| 2000/2000 [04:28<00:00,  7.45it/s, v_num=pcsz]  
Validation: 0it [00:00, ?it/s]
Epoch 5:  60%|██████    | 1202/2000 [02:42<01:47,  7.40it/s, v_num=pcsz]  